In [1]:
import sys
import re
import math
import requests
import enlighten

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

url = 'http://200.152.38.155/CNPJ/'
formato_data = '%Y%m%d'
PATH_BD = 'data/CNPJ_full.db'

In [ ]:
    

list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
tamanho = len(filtered)
MANAGER = enlighten.get_manager()
bara_lista = MANAGER.counter(total=tamanho, desc='Arquivos', unit='Arquivos')


print("")
print(f"Fazendo download dos {tamanho} arquivos encontrados.", flush=False)
print("")


for indice, elemento in enumerate(filtered):
    bara_lista.update()
    
    r = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
    if r.status_code == requests.codes.ok:
        assert r.status_code == 200, r.status_code
        dlen = int(r.headers.get('Content-Length', '0')) or None
        barra_download = MANAGER.counter(color = 'green', desc= f'Arquivo {indice + 1} de {tamanho}', 
                             total = dlen and math.ceil(dlen / 2 ** 20), 
                             unit = 'MiB', leave = False)
        with barra_download as ctr, \
             open("zips/" + elemento, 'wb', buffering = 2 ** 24) as f:
            for chunk in r.iter_content(chunk_size = 2 ** 20):
                # print(chunk[-16:].hex().upper())
                print(f'\r{elemento} ------ Está sendo baixado!!!', end='', flush=True)
                sys.stdout.flush()
                f.write(chunk)
                ctr.update()
        
    else:
        r.raise_for_status()
print(f'\r Download Concluido!!!', end='', flush=True)
MANAGER.stop()

In [2]:
list_estabelecimentos = [
"cnpj", 
"cnpj_ordem", 
"cnpj_dv", 
"matriz_filial",
"nome",
"situacao", 
"data_situacao", 
"motivo_situacao", 
"cidade_exterior", 
"pais", 
"inicio_atividade", 
"cnae_fiscal", 
"cnae_secundario", 
"tipo_logradouro", 
"logradouro", 
"numero", 
"complemento", 
"bairro", 
"cep", 
"uf", 
"municipio", 
"ddd_1", 
"telefone_1", 
"ddd_2", 
"telefone_2", 
"ddd_fax", 
"numero_fax", 
"email", 
"situacao_especial", 
"data_situacao_especial"
]

In [3]:
df = pd.read_csv("zips/" +'K3241.K03200Y0.D10710.ESTABELE.zip', 
                 compression='zip',
                 header=None, 
                 sep=';', 
                 quotechar='"', 
                 encoding = "ISO-8859-1", 
                 low_memory=False,
                 dtype = {'cnpj': str, 'cnpj_ordem': str, 'cnpj_dv': str},
                 names=list_estabelecimentos)
df

,cnpj,cnpj_ordem,cnpj_dv,matriz_filial,nome,situacao,data_situacao,motivo_situacao,cidade_exterior,pais,inicio_atividade,cnae_fiscal,cnae_secundario,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,municipio,ddd_1,telefone_1,ddd_2,telefone_2,ddd_fax,numero_fax,email,situacao_especial,data_situacao_especial
0,26638409,0001,21,1,CANTINHO DA LILICA,2,20161130,0,NaN,NaN,20161130,9602502,9602501,RUA,maria olinda silva,119,APT 202,centro,33200000,MG,5425,31,92667948,NaN,NaN,NaN,NaN,elissouzap@hotmail.com,NaN,NaN
1,26638420,0001,91,1,NOBRE MADEIRAS DO BRASIL,4,20210212,63,NaN,NaN,20161130,4744001,"7739099,4754701,2391502,7731400,7732201,331479...",RUA,ITUMBIARA,1143,QUADRA 155;LOTE 25,CIDADE JARDIM,74413120,GO,9373,62,39881414,NaN,NaN,NaN,NaN,fmix.contabilidade@gmail.com,NaN,NaN
2,26638434,0001,05,1,TECNODONTOMEDICAL,8,20161216,1,NaN,NaN,20161130,3319800,4321500,RUA,K,385,CASA ;CASA ;CASA ;CASA,SAO CONRADO,49043236,SE,3105,79,98704380,NaN,NaN,NaN,NaN,alex28ndre@gmail.com,NaN,NaN
3,26638446,0001,30,1,NaN,4,20210303,63,NaN,NaN,20161130,9700500,NaN,AVENIDA,PEDRA BRANCA,13,NaN,VILA SAO JOSE I,65130000,MA,849,98,32649607,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26638459,0001,09,1,NaN,2,20161130,0,NaN,NaN,20161130,2542000,2599301,RUA,MONTOJO,115,APT 202 B,QUINTA DA PAINEIRA,03152160,SP,7107,11,99999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6417507,42746009,0001,96,1,LOJA HUGOSTORE,2,20210716,0,NaN,NaN,20210716,4755503,"4781400,4763602,4782202,4783102,4783101",RUA,MIGUEL RUFINO,18,NaN,ALTO DO TRIANGULO,59515000,RN,1615,84,91515949,NaN,NaN,NaN,NaN,TRINDADECONTABILIDADE123@GMAIL.COM,NaN,NaN
6417508,42746019,0001,21,1,BACCHIN FONOAUDIOLOGIA,2,20210716,0,NaN,NaN,20210716,8650006,NaN,AVENIDA,S MIGUEL,4710,NaN,JARDIM COTINHA,03870100,SP,7107,11,85842837,NaN,NaN,NaN,NaN,LETICIABBACCHIN@GMAIL.COM,NaN,NaN
6417509,42746030,0001,91,1,NaN,2,20210716,0,NaN,NaN,20210716,8219999,NaN,AVENIDA,CONCEICAO,933,APT 23,CARANDIRU,02072001,SP,7107,11,21882878,NaN,NaN,NaN,NaN,FILIPELACERDA2661@GMAIL.COM,NaN,NaN
6417510,42746041,0001,71,1,CLINICA SORRIA BOM JESUS DA LAPA,2,20210716,0,NaN,NaN,20210716,8630504,NaN,AVENIDA,ALMIRANTE BEIRUTTI,272,NaN,CENTRO,47600000,BA,3377,77,98289257,NaN,NaN,NaN,NaN,CONTABEDCON@HOTMAIL.COM,NaN,NaN


In [ ]:
df['data_situacao']= df['data_situacao'].replace(0, np.nan)

df['data_situacao'] = pd.to_datetime(df['data_situacao'], format=formato_data)
df['inicio_atividade'] = pd.to_datetime(df['inicio_atividade'], format=formato_data)
df['data_situacao_especial'] = pd.to_datetime(df['data_situacao_especial'], format=formato_data)

In [ ]:
import sqlite3

connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

sql = """CREATE TABLE IF NOT EXISTS estabelecimentos 
                    (cnpj                   VARCHAR (8),
                    cnpj_ordem             VARCHAR (4),
                    cnpj_dv                VARCHAR (2),
                    matriz_filial          INTEGER,
                    nome                   VARCHAR,
                    situacao               INTEGER,
                    data_situacao          DATE,
                    motivo_situacao        VARCHAR,
                    cidade_exterior        VARCHAR,
                    pais                   INTEGER,
                    inicio_atividade       DATE,
                    cnae_fiscal            VARCHAR,
                    cnae_secundario        VARCHAR,
                    tipo_logradouro        VARCHAR,
                    logradouro             VARCHAR,
                    numero                 VARCHAR,
                    complemento            VARCHAR,
                    bairro                 VARCHAR,
                    cep                    VARCHAR (8),
                    uf                     VARCHAR (2),
                    municipio              INTEGER,
                    ddd_1                  VARCHAR,
                    telefone_1             VARCHAR (11),
                    ddd_2                  INTEGER,
                    telefone_2             VARCHAR (11),
                    ddd_fax                INTEGER,
                    numero_fax             VARCHAR (11),
                    email                  VARCHAR,
                    situacao_especial      VARCHAR,
                    data_situacao_especial DATE 
                    )
                """


con.execute('DROP TABLE IF EXISTS estabelecimentos') 
con.execute(sql)

# for percorrendo todos os arquivos de estavelecimentos
# Fazer o vacum

df.to_sql(name = 'estabelecimentos', con = connection, if_exists = 'append', index = False)

In [ ]:
back = pd.read_sql('select * from estabelecimentos', connection)

back

In [ ]:

dtypes = np.dtype(
    [
        ("a", str),
        ("b", int),
        ("c", float),
        ("d", np.datetime64),
        
    ]
)
df = pd.DataFrame(np.empty(0, dtype=dtypes))

In [ ]:
encoding_list = ['ascii', 'big5', 'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437', 'cp500', 'cp720', 'cp737'
                 , 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862'
                 , 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950'
                 , 'cp1006', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254'
                 , 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'euc_jp', 'euc_jis_2004', 'euc_jisx0213', 'euc_kr'
                 , 'gb2312', 'gbk', 'gb18030', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2'
                 , 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1', 'iso8859_2'
                 , 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9'
                 , 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'johab'
                 , 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2'
                 , 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'utf_32'
                 , 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 'utf_8', 'utf_8_sig']

for encoding in encoding_list:
    worked = True
    try:
        df = pd.read_csv('K3241.K03200Y0.D10710.ESTABELE.zip', compression='zip',header=None, sep=';', quotechar='"', encoding=encoding)
    except:
        worked = False
    if worked:
        print(encoding, ':\n', df.head())

In [ ]:
import math
import requests, enlighten

url = 'http://200.152.38.155/CNPJ/F.K03200$W.SIMPLES.CSV.D10710.zip'
fname = 'F.K03200$W.SIMPLES.CSV.D10710.zip'

# Should be one global variable
MANAGER = enlighten.get_manager()

r = requests.get(url, stream = True)
assert r.status_code == 200, r.status_code
dlen = int(r.headers.get('Content-Length', '0')) or None

with MANAGER.counter(color = 'green', total = dlen and math.ceil(dlen / 2 ** 20), unit = 'MiB', leave = False) as ctr, \
     open(fname, 'wb', buffering = 2 ** 24) as f:
    for chunk in r.iter_content(chunk_size = 2 ** 20):
        # print(chunk[-16:].hex().upper())
        f.write(chunk)
        ctr.update()

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
tamanho = len(filtered)
for indice, elemento in tqdm(enumerate(filtered)):
        resposta = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
        if resposta.status_code == requests.codes.ok:
            total_size = int(resposta.headers.get("Content-Length", 0))
            block_size = 1024

            print(f"Downloading {elemento}")

            with tqdm(total=total_size,file=sys.stdout) as pbar:
                with open(elemento, "wb") as f:
                    for data in resposta.iter_content(block_size):
                        pbar.set_description(f"Processing {indice +1}")
                        pbar.update(len(data))
                        f.write(data)
        else:
                resposta.raise_for_status()


In [ ]:
from tqdm import tqdm
from time import sleep

In [ ]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

In [ ]:
donloads(url, list_urls[1])

In [ ]:
def donloads(url, arquivo):
    resposta = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(resposta.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in resposta.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:

req = requests.get(url, stream=True, headers={"Accept-Encoding": None})

In [ ]:
req.links

In [ ]:
url = 'http://200.152.38.155/CNPJ/'

In [ ]:
def baixa_arquivo(url, endereco):
    # faz a requisição 
    resposta = requests.get(url)
    if resposta.status_code == requests.codes.ok:
        with open('endereco', 'wb')as novo_arquivo:
            novo_arquivo.write(resposta.content)
        print("Download finalizado, salvo em: ()".format(endereco))
    else:
        respósta.raise_for_status()

In [ ]:
baixa_arquivo(url, 'teste.html')

In [ ]:
data = str(urlopen(URL_BASE_RFB).read(), encoding='utf8')

    # Pega todas as urls
    urls = re.findall(r'href=[\'"]?([^\'" >]+)', data)

In [ ]:
resposta = requests.get(url).text

In [ ]:
print(resposta.text)

In [ ]:
import re

In [ ]:
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)

In [ ]:
list_urls

In [ ]:
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip","pdf"], list_urls))
filtered

In [ ]:
for indice, elemento in tqdm(enumerate(filtered)):
    print (f"{indice:<10}{elemento}")

In [ ]:
def donloads(url, arquivo):
    req = requests.get(url + arquivo, stream=True, headers={"Accept-Encoding": None})
    if resposta.status_code == requests.codes.ok:
        total_size = int(req.headers.get("Content-Length", 0))
        block_size = 1024

        print(f"Downloading {arquivo}")

        t = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(arquivo, "wb") as f:
            for data in req.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
    else:
        resposta.raise_for_status()

In [ ]:
for i in list_urls:
    if i.split(".")[-1] in ["zip","pdf"]:
        donloads(url, i)

In [ ]:
donloads(url, list_urls[1])